# Gentle introduction to social networks in Python

### Prof. Jose Manuel MAGALLANES.
* Professor at Pontificia Universidad Católica del Peru
* Senior Data Scientist at eScience Institute, University of Washington

_______


## CONTENTS:

### 1. Network from dataframes


These data frames have been previously prepared, based on the data from [50 influential people in Seattle tech to follow on Twitter](https://www.builtinseattle.com/2018/08/06/50-seattle-tech-twitter-accounts-to-follow). The file can be seen [here](https://docs.google.com/spreadsheets/d/e/2PACX-1vTpWvtfphnO9eRYwmEyxVAxxo1KEsdpK6sK6q5uhJn5x2QtB-eGiso8ibpF16NaHAers4wDIHkvBo64/pubhtml), and includes 3 sheets:

* Network as two columns representing edges.
* Network as adjacency matrix.
* Attributes of the Network nodes.

In [1]:
fileLink='https://github.com/eScience-UW/gentleIntro_networks/raw/main/data/seattleTop.xlsx'

import pandas as pd

edges=pd.read_excel(io=fileLink,sheet_name="edges")
adjacency=pd.read_excel(io=fileLink,sheet_name="adjacency",index_col=0)# index_col!
attributes=pd.read_excel(io=fileLink,sheet_name="attributes")

Use the previous dataframes to create the network:

* From the edges:

In [2]:
import networkx as net # package needed

EliteNet =net.from_pandas_edgelist(edges)


In [3]:

EliteNet,len(EliteNet.nodes),len(EliteNet.edges)

(<networkx.classes.graph.Graph at 0x288032a1070>, 46, 393)

* From the adjacency:

In [4]:

EliteNet = net.Graph(adjacency)


In [5]:
EliteNet,len(EliteNet.nodes),len(EliteNet.edges)

(<networkx.classes.graph.Graph at 0x28803403220>, 46, 393)

In [8]:
EliteNet.edges(data=True)

EdgeDataView([('rachelerman', 'mattmcilwain', {'weight': 1}), ('rachelerman', 'DaveParkerSEA', {'weight': 1}), ('rachelerman', 'toddbishop', {'weight': 1}), ('rachelerman', 'ashannstew', {'weight': 1}), ('rachelerman', 'LeslieFeinzaig', {'weight': 1}), ('rachelerman', 'akipman', {'weight': 1}), ('rachelerman', 'matt_oppy', {'weight': 1}), ('rachelerman', 'gilbert', {'weight': 1}), ('rachelerman', 'juliesandler', {'weight': 1}), ('rachelerman', 'BradSmi', {'weight': 1}), ('rachelerman', 'crashdev', {'weight': 1}), ('rachelerman', 'ShaunaCausey', {'weight': 1}), ('rachelerman', 'john_gabbert', {'weight': 1}), ('rachelerman', 'moniguzman', {'weight': 1}), ('rachelerman', 'mattmday', {'weight': 1}), ('rachelerman', 'Rich_Barton', {'weight': 1}), ('rachelerman', 'daryn', {'weight': 1}), ('rachelerman', 'lovelletters', {'weight': 1}), ('rachelerman', 'etzioni', {'weight': 1}), ('rachelerman', 'MissDestructo', {'weight': 1}), ('rachelerman', 'heatherredman', {'weight': 1}), ('rachelerman', 'd

### 2. Adding Attributes

Attributes are added using dictionaries _{node_name:attribute_value}_:

In [ ]:
# create dictionaries
dictAttribute_male=dict(zip(attributes.name,attributes.male))
dictAttribute_followers=dict(zip(attributes.name,attributes.followers))

In [ ]:
dictAttribute_male

In [ ]:
# set attribute with the dictionaries
net.set_node_attributes(EliteNet, dictAttribute_male,'male')
net.set_node_attributes(EliteNet, dictAttribute_followers,'followers')

In [ ]:
## see them
EliteNet.nodes(data=True)

In [ ]:
EliteNet.edges()

In [ ]:
maleValues=net.get_node_attributes(EliteNet,'male').values()
colors_for_nodes=['green' if attr==1 else 'red' for attr in maleValues]

In [ ]:
net.draw(EliteNet,
        with_labels=True,
        node_color=colors_for_nodes)

## 3. Exploring the Network

* Connectedness: A network is  “connected” if there exists a _path_ between any pair of nodes (undirected networks).

In [ ]:
net.is_connected(EliteNet)

* Density: from 0 to 1, where 1 makes it a ‘complete’ network: there is a link between every pair of nodes.

In [ ]:
net.density(EliteNet) 

* Diameter: When two vertices are connected, one can reach the other using multiple egdes. The geodesic is the shorthest path between two connected vertices. Then, the diameter, is the maximum geodesic in a network.

In [ ]:
net.diameter(EliteNet)

* Assortativity: it is a measure to see if nodes are connecting to other nodes similar to themselves. Closer to 1 means higher assortativity, closer to -1 diassortativity; while 0 is no assortativity.

a.  Degree assortativity: tendency for highly connected nodes to be connected among themselves.

In [ ]:
net.degree_assortativity_coefficient(EliteNet)

b.  Categorical assortativity: tendency for nodes with other nodes sharing the same category.

In [ ]:
net.attribute_assortativity_coefficient(EliteNet,'male')

c. Assortativity (numerical): 

In [ ]:
net.numeric_assortativity_coefficient(EliteNet,'followers')

## 4. Exploration of Network nodes

- The eigenvector of a vertex will tell you how well connected is a vertex; that is, vertices with the highest values are considered the most influential as they are connected to vertices that are also well connected.

- The closeness of a vertex will tell you how close is a vertex to every other vertex. A vertex with high closeness can share information faster than the rest.

- The betweeness of a vertex will tell you how critical is a vertex to connect vertex that are not connected directly.

In [ ]:
# Computing centrality measures:

eigen=net.eigenvector_centrality(EliteNet)

clos=net.closeness_centrality(EliteNet)

betw=net.betweenness_centrality(EliteNet,normalized=True)


In [ ]:
# the dataframe of centralities

DFCentrality=pd.DataFrame(dict(Eigenvector = eigen,
                               Closeness = clos,
                               Betweenness = betw)) 


In [ ]:
DFCentrality.reset_index(drop=False,names='person',inplace=True)
DFCentrality

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(7,7))

ax.scatter(DFCentrality.Betweenness, DFCentrality.Closeness,s=(DFCentrality.Eigenvector+1)**35, alpha=0.5)

valsForAnnotate=zip(DFCentrality['person'],DFCentrality['Betweenness'],DFCentrality['Closeness'])

for name,coordX,coordY in valsForAnnotate:
    ax.annotate(name, (coordX,coordY),alpha=0.5,size=7)
    
plt.title("scatterplot (size for Eigenvector of node)")
plt.xlabel("Betweenness")
plt.ylabel("Closeness")
plt.show()

The previous results tells us that two people are salient:

In [ ]:
# Determine the hub name:

labelsHubs={n:n for n in DFCentrality.nlargest(2, 'Eigenvector')['person']}
labelsHubs

let's highlight those guys:

In [ ]:
# prepare to plot:
fig, ax = plt.subplots(figsize=(7,7))

## positions of the nodes
pos=net.spring_layout(EliteNet)

#edges
net.draw_networkx_edges(EliteNet,pos,edge_color='grey',alpha=0.2)
# make the hub salient
net.draw_networkx_nodes(EliteNet,pos,nodelist=labelsHubs,node_size=200,node_color='r')
# label the hubs
net.draw_networkx_labels(EliteNet,pos,labels=labelsHubs,font_size=8,font_color='blue')

plt.show()

## 5. Communities

In [ ]:
RandomNet=net.erdos_renyi_graph(len(EliteNet.nodes), p=0.5)

* Transitivity: How probable is that two nodes with a common connection, are also connected.

In [ ]:
net.transitivity(RandomNet)

In [ ]:
net.transitivity(EliteNet)

* Partition

Set of nodes that belong to a group. The hard problems is to decide what node goes where:

In [ ]:
# edges that will partition the net
edgesThePartition=net.minimum_edge_cut(EliteNet)
type(edgesThePartition), edgesThePartition

Let's plot:

In [ ]:
# labels for the nodes that were part of the edge removed

byNodes=set(sum(net.minimum_edge_cut(EliteNet),())) # flattening the set
labelsBye={n:n for n in byNodes} # a dictionary

## see
labelsBye

In [ ]:
EliteNet_cut=EliteNet.copy() 

EliteNet_cut.remove_edges_from(edgesThePartition)


fig, ax = plt.subplots(figsize=(10,10))
pos=net.spring_layout(EliteNet) # positions for the original net
net.draw(EliteNet_cut,pos=pos,node_color='yellow')  # plotting the cuts 
net.draw_networkx_labels(EliteNet,pos,
                         labels=labelsBye,font_size=12,font_color='red'); # labelling



The **girvan_newman** algorithm is based on creating partitions using the centrality of nodes to create partitions too. We can create something similar to the last result like this:

In [ ]:
# generator of partitions
partition_girvanNewman_all = net.community.girvan_newman(EliteNet) 
# the first partition
partition_girvanNewman_first = list(set(sorted(c)) for c in next(partition_girvanNewman_all)) 

Replotting the previous:

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

pos = net.spring_layout(EliteNet)

cmap = plt.get_cmap('Accent', len(partition_girvanNewman_first)) # amount of colors

net.draw_networkx_edges(EliteNet, pos, alpha=0.2) # all edges
net.draw_networkx_labels(EliteNet,pos,
                         labels=labelsBye,font_size=6,font_color='red'); # labelling


for color, nodes in enumerate(partition_girvanNewman_first): # position , nodeset
    
    net.draw_networkx_nodes(EliteNet, pos, nodes, node_size=50,
                           node_color=[cmap.colors[color]])

A more complex algorithm is the [Louvain](https://perso.uclouvain.be/vincent.blondel/research/louvain.html): 

In [ ]:
partition_louvain=net.community.louvain_communities(EliteNet)

In [ ]:
###

fig, ax = plt.subplots(figsize=(10,10))
pos = net.spring_layout(EliteNet,k=3)
cmap = plt.get_cmap('Accent', len(partition_louvain))
net.draw_networkx_edges(EliteNet, pos, alpha=0.2)
for color, nodes in enumerate(partition_louvain):
    
    net.draw_networkx_nodes(EliteNet, pos, nodes, node_size=100,
                           node_color=[cmap.colors[color]])

In [ ]:
partition_girvanNewman_all = list(net.community.girvan_newman(EliteNet))
pd.DataFrame([partition_girvanNewman_all[len(partition_louvain)],
              partition_girvanNewman_all[len(partition_louvain)-1],
              partition_girvanNewman_all[len(partition_louvain)-2]]).T

* Modularity:

If we get positive values (being 1 the top value), we could consider there are good community structures ([wiki](https://en.wikipedia.org/wiki/Modularity_(networks))). The higher modularity the connections between the nodes are dense  within a partition but sparse  between nodes in different partitions.

In [ ]:
a_partition_girvanNewman=partition_girvanNewman_all[len(partition_louvain)-2]

net.community.modularity(EliteNet,a_partition_girvanNewman)

In [ ]:
net.community.modularity(EliteNet,partition_louvain)

## Exporting

In [ ]:
from collections import ChainMap


GnNn_attr=dict(ChainMap(*[{node:a for node in b} for a,b in enumerate(a_partition_girvanNewman)]))
GnNn_attr

In [ ]:
Ln_attr=dict(ChainMap(*[{node:a for node in b} for a,b in enumerate(partition_louvain)]))
Ln_attr

In [ ]:
net.set_node_attributes(EliteNet, GnNn_attr,'Girvan_Newman_partition')
net.set_node_attributes(EliteNet, Ln_attr,'Louvain_partition')

In [ ]:
net.write_graphml(EliteNet, "EliteNet_py.graphml",encoding='utf-8')